In [1]:
import imp
from deeprankcore.utils import exporters
from deeprankcore import trainer
imp.reload(exporters)
imp.reload(trainer)

/var/folders/js/v54t24f101585t5fkkx3qpkh0000gn/T/ipykernel_74237/1054045491.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'deeprankcore.trainer' from '/Users/giuliacrocioni/Desktop/docs/eScience/projects/3D-vac/deeprank-core/deeprankcore/trainer.py'>

In [2]:
from deeprankcore.dataset import GraphDataset
from deeprankcore.domain import (edgestorage as Efeat, nodestorage as Nfeat,
                                targetstorage as targets)
from deeprankcore.neuralnets.ginet import GINet

dataset_train = GraphDataset(
    hdf5_path = 'tests/data/hdf5/train.hdf5',
    target = targets.BINARY
)

dataset_val = GraphDataset(
    hdf5_path = 'tests/data/hdf5/valid.hdf5',
    target = targets.BINARY
)

dataset_test = GraphDataset(
    hdf5_path = 'tests/data/hdf5/test.hdf5',
    target = targets.BINARY,
)

my_trainer = trainer.Trainer(
    GINet,
    dataset_train,
    dataset_val,
    dataset_test
)   

   ['tests/data/hdf5/train.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 181.09it/s, mol=train.hdf5]
   ['tests/data/hdf5/valid.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 398.74it/s, mol=valid.hdf5]
   ['tests/data/hdf5/test.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 344.47it/s, mol=test.hdf5]


100%|██████████| 4/4 [00:00<00:00, 11.27it/s]


In [3]:
my_trainer.train(nepoch=3, validate=True)

/Users/giuliacrocioni/Desktop/docs/eScience/projects/3D-vac/deeprank-core/deeprankcore/utils/metrics.py:298: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['phase', 'entry', 'output'], dtype='object')]

  self.df.to_hdf(


In [4]:
my_trainer.test()

100%|██████████| 4/4 [00:00<00:00, 13.03it/s]


In [9]:
import pandas as pd
df_testing = pd.read_hdf('output/output_exporter.hdf5', key='testing')
df_training = pd.read_hdf('output/output_exporter.hdf5', key='training')

In [10]:
assert len(df_training.epoch.unique()) == 4
assert len(df_testing.epoch.unique()) == 1
assert df_training[df_training.phase == 'training'].groupby(['epoch'], as_index=False).count().phase.unique() == 44
assert df_training[df_training.phase == 'validation'].groupby(['epoch'], as_index=False).count().phase.unique() == 2
assert df_testing[df_testing.phase == 'testing'].groupby(['epoch'], as_index=False).count().phase.unique() == 4
assert df_training[df_training.phase == 'training'].shape[0] == 44*4
assert df_training[df_training.phase == 'validation'].shape[0] == 2*4
assert df_testing[df_testing.phase == 'testing'].shape[0] == 4